In [39]:
import pandas as pd
import numpy as np
import time

In [40]:
# # data = pd.read_csv("D:/Udler downloads/train.csv")
data = pd.read_csv("../../../datasets/Churn_Modelling.csv")

In [41]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [42]:
data.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [43]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [44]:
DATA_PATH = "../../../datasets/Churn_Modelling.csv"

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [46]:
def load_and_standardize_data(path):
    # read in from csv
    df = pd.read_csv(path, sep=',')
    # replace nan with -99
    # df = df.fillna(-99)
    y = df['Exited']
    df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
       'Gender','Exited'],axis=1,inplace=True)
    xtrain,xtest,ytrain,ytest = train_test_split(df,y,stratify = y,test_size = 0.3,random_state = 4)
    # df = df.values.reshape(-1, df.shape[1]).astype('float32')
    # randomly split

    # standardize values
#     scaler = preprocessing.StandardScaler()
    scaler=preprocessing.MinMaxScaler()
    xtrain = scaler.fit_transform(xtrain)
    xtest= scaler.transform(xtest)
    return xtrain,xtest,scaler

In [47]:
from torch.utils.data import Dataset, DataLoader
class DataBuilder(Dataset):
    def __init__(self, path, train=True):
        self.X_train, self.X_test, self.standardizer = load_and_standardize_data(DATA_PATH)
        if train:
            # self.x = torch.from_numpy(self.X_train)
            self.x = torch.FloatTensor(self.X_train)
            self.len=self.x.shape[0]
        else:
            # self.x = torch.from_numpy(self.X_test)
            self.x = torch.FloatTensor(self.X_test)
            self.len=self.x.shape[0]
        del self.X_train
        del self.X_test
    def __getitem__(self,index):
        return self.x[index]
    def __len__(self):
        return self.len

In [48]:
traindata_set=DataBuilder(DATA_PATH, train=True)
testdata_set=DataBuilder(DATA_PATH, train=False)

trainloader=DataLoader(dataset=traindata_set,batch_size=20000)
testloader=DataLoader(dataset=testdata_set,batch_size=20000)

In [49]:
type(trainloader.dataset.x), type(testloader.dataset.x)

(torch.Tensor, torch.Tensor)

In [50]:
trainloader.dataset.x.shape, testloader.dataset.x.shape

(torch.Size([7000, 8]), torch.Size([3000, 8]))

In [51]:
trainloader.dataset.x

tensor([[0.8700, 0.2162, 0.9000,  ..., 1.0000, 1.0000, 0.3430],
        [0.8600, 0.2568, 1.0000,  ..., 1.0000, 0.0000, 0.6317],
        [0.6340, 0.2432, 0.3000,  ..., 1.0000, 1.0000, 0.9344],
        ...,
        [0.3860, 0.2568, 0.3000,  ..., 1.0000, 1.0000, 0.3946],
        [0.4180, 0.3243, 0.7000,  ..., 1.0000, 1.0000, 0.9504],
        [0.7880, 0.3378, 1.0000,  ..., 0.0000, 1.0000, 0.1211]])

In [52]:
testloader.dataset.x

tensor([[0.7860, 0.2973, 0.8000,  ..., 1.0000, 0.0000, 0.4745],
        [0.2760, 0.0541, 0.6000,  ..., 1.0000, 1.0000, 0.3320],
        [0.5320, 0.3378, 0.3000,  ..., 1.0000, 0.0000, 0.0938],
        ...,
        [0.5060, 0.1757, 0.8000,  ..., 1.0000, 1.0000, 0.8498],
        [0.5540, 0.3514, 0.6000,  ..., 0.0000, 0.0000, 0.1765],
        [0.5280, 0.2432, 0.1000,  ..., 1.0000, 0.0000, 0.7308]])

In [53]:
latent_dims = 2
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

## WNN Functions

In [54]:
1/np.sqrt(3)

0.5773502691896258

In [55]:


def ggw(x):
    return torch.sin(3.0 * x)+ torch.sin(0.3 * x)+  torch.sin(0.03 * x)

def morlet_wavelet(x):
#     print(x)
    return (torch.cos(1.75 * x)) * torch.exp(-x**2/2)

def gaussian_wavelet(x,sigma = 1, omega = 0.5 ):
    return torch.exp(-0.5 * (x ** 2) / (sigma** 2)) * torch.cos(omega * x)

def mexhican_hat_wavelet(x, sigma=2, omega=0.577):
    return (torch.pi** (-0.25)*sigma*omega)*(1-x**2)*(torch.exp(-0.5 * (-x**2))) 

def shannon(x):
    return torch.sin(torch.pi *(x-0.5)) - torch.sin(2*(x-0.5) * torch.pi) *(1/torch.pi*(x-0.5))

In [56]:
#shannon(torch.tensor(3.4))

In [57]:
import math
import torch
import torch.nn as nn
import numpy as np

In [58]:
import random

In [59]:
rx=torch.randn(23,50)
weights = torch.Tensor(rx)
print("weights_Encoder",weights)
print("weights_Encoder_shape",weights.shape)

weights_Encoder tensor([[-0.2587, -1.9871, -1.8841,  ..., -0.6392,  2.0119,  0.8981],
        [-0.0707,  0.4204,  0.7223,  ..., -1.6554, -0.6218,  1.3797],
        [-0.3862,  1.1394, -1.1415,  ..., -0.2200, -0.4270,  0.1381],
        ...,
        [-0.3131, -0.2417, -1.5116,  ..., -0.3396,  1.2693, -0.4109],
        [-1.0586,  0.7272,  0.1517,  ...,  0.6031,  0.0973, -0.4445],
        [ 0.8191,  0.4164, -1.0945,  ..., -0.0949, -0.2919,  1.3822]])
weights_Encoder_shape torch.Size([23, 50])


In [60]:
weights.t().shape

torch.Size([50, 23])

In [61]:
    x = torch.randn(50)
    torch.Tensor(x).reshape(1,-1)

tensor([[-1.0087,  0.9747,  0.5334,  2.0259, -0.0549, -0.0174,  0.2791, -0.3582,
          1.1871, -0.5368,  2.2079, -0.6975,  0.1586,  0.1188, -1.0348,  0.2127,
          0.8974,  0.1196,  0.0820,  0.2775,  0.1936,  0.7112,  0.4225,  0.5149,
         -1.3638,  1.7272,  2.0074, -0.7556,  0.3058, -0.6672, -0.2793,  0.0289,
          1.3275,  1.7348, -1.2469,  0.6087,  0.4441, -1.1407,  0.9608,  0.7155,
         -0.7672,  2.3813,  1.3094,  0.0919,  0.3331, -1.5810,  1.0645,  0.1478,
         -0.4754,  0.5659]])

In [62]:
x[0]

tensor(-1.0087)

In [63]:
from torch.nn.functional import linear

class WnnLayer(nn.Module):

    def __init__(self, size_in, size_out):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out

        #random_seed = 1
        # torch.manual_seed(random_seed)

        rx=torch.randn(size_in, size_out)
        weights = torch.Tensor(rx)
#         print('weights',weights.shape)

        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
#         self.lr=lr
        # print(self.weights)
        # print((self.weights).shape)
        bias = torch.Tensor(size_out)
        self.bias = nn.Parameter(bias)
        drx=torch.randn(self.size_out)
        self.dilation = torch.Tensor(drx)

        self.dilation.requires_grad=True
        self.dilation=nn.Parameter(self.dilation)
        trx=torch.randn(self.size_out)
        self.translation = torch.Tensor(trx)

        self.translation.requires_grad=True
        self.translation=nn.Parameter(self.translation)

        # initialize weights and biases
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weights)
#         self.lr=lr
        # nn.init.uniform_(self.bias, -bound, bound)  # bias init
        # self.dilation = nn.init.uniform_(self.dilation)
        # self.translation = nn.init.uniform_(self.translation)

    def forward(self, x):
#         print('x',x.shape)
        x1=x.reshape(1,-1)
#         print('x1.shape',x1.shape)
#         print(x1.shape,self.weights.t().shape)
        w_times_x= torch.mm(x, self.weights)
# #         print('w_timesx.shape',w_times_x.shape)

        mean=torch.mean(self.dilation)
        std=torch.std(self.dilation)
        #self.dilation = torch.nn.Parameter(torch.div(torch.sub(self.dilation,mean),std))


        mean=torch.mean(self.translation)
        std=torch.std(self.translation)
        #self.translation = torch.nn.Parameter(torch.div(torch.sub(self.translation,mean),std))

        ###################################################print(self.translation)

        #hid = torch.div(torch.sub(w_times_x , self.dilation_a),self.translation_b)
#         print(w_times_x.shape,self.dilation.shape)

        self.dilation_a=self.dilation.reshape(1,-1)
#         print('dilation.shape',self.dilation.shape)
        self.translation_b=self.translation.reshape(1,-1)
#         print('translation.shape',self.translation.shape)

        hid = torch.div(torch.sub(w_times_x , self.dilation_a),self.translation_b)
#         hid.reshape(-1,1)
#         print('hid',hid.shape)

        hid[hid!= hid]=0.0
#         print(hid.shape)
        return hid
        # return torch.add(w_times_x, self.bias)  # w times x + b

In [64]:
from torch.nn.functional import linear

class WnnLayer1(nn.Module):

    def __init__(self, size_in, size_out):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out
        random_seed = 1
        #torch.manual_seed(random_seed)
        rx=torch.randn(size_out, size_in)
        weights = torch.Tensor(rx)
       # print("weights_Decoder",weights)
#         print("weights_Decoder_shape",weights.shape)
        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
        # print(self.weights)
        # print((self.weights).shape)
        bias = torch.Tensor(size_out)
        self.bias = nn.Parameter(bias)
        drx=torch.randn(self.size_out)
        self.dilation = torch.Tensor(drx)
        self.dilation.requires_grad=True
        self.dilation=nn.Parameter(self.dilation)
        trx=torch.randn(self.size_out)
        self.translation = torch.Tensor(trx)
        self.translation.requires_grad=True
        self.translation=nn.Parameter(self.translation)
#         self.lr=lr

        # initialize weights and biases
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weights)


    def forward(self, x):
#         print('x.shape',x.shape)

        x1=x.reshape(1,-1)
#         print('x1 and weights shapes: ',x1.shape,self.weights.t().shape)
        w_times_x= torch.mm(x, self.weights.t())
#         print('w_times_x.shape',w_times_x.shape)

#         print(f'w_times_x and dilation shapes: ',w_times_x.shape,self.dilation.shape)
        mean=torch.mean(self.dilation)
        std=torch.std(self.dilation)
        #self.dilation = torch.nn.Parameter(torch.div(torch.sub(self.dilation,mean),std))


        mean=torch.mean(self.translation)
        std=torch.std(self.translation)
        #self.translation = torch.nn.Parameter(torch.div(torch.sub(self.translation,mean),std))


        self.dilation_a=self.dilation.reshape(1,-1)
#         print('dilation.shape',self.dilation.shape)

        self.translation_b=self.translation.reshape(1,-1)
#         print('translation.shape',self.translation.shape)


        hid = torch.div(torch.sub(w_times_x , self.dilation_a),self.translation_b)
#         print('hid.shape',hid.shape)

#         print(f'hidden values shapes: ',hid.shape)
        return hid
        # return torch.add(w_times_x, self.bias)  # w times x + b

## Encoder

In [65]:
## GGW

class Encoder2(nn.Module):
    def __init__(self,input_no,latent_dims,fn,H2 = 10,H3=15):
        super(Encoder2,self).__init__()
        self.linear1 = WnnLayer(input_no,H2)
        self.binorm1 = nn.BatchNorm1d(num_features = H2)
        self.linear2 = WnnLayer(H2,H3)
        self.binorm2 = nn.BatchNorm1d(num_features = H3)
        self.linear3 = nn.Linear(H3,latent_dims)
        self.binorm3= nn.BatchNorm1d(num_features = latent_dims)
        self.linear4= nn.Linear(H3 , latent_dims)
        self.binorm4= nn.BatchNorm1d(num_features = latent_dims)
        self.relu =nn.ReLU()
        self.fn=fn
        #act=['morlet','ggw','mexi','shan','gaus']
    def forward(self, x):
        if self.fn=='ggw':
            lin1 = ggw(self.binorm1(self.linear1(x)))
            lin2 = ggw(self.binorm2(self.linear2(lin1)))
            Mu = ggw(self.binorm3(self.linear3(lin2)))
            sigma = ggw(self.binorm4(self.linear4(lin2)))
        if self.fn=='morlet':
            lin1 = morlet_wavelet(self.binorm1(self.linear1(x)))
            lin2 = morlet_wavelet(self.binorm2(self.linear2(lin1)))
            Mu = morlet_wavelet(self.binorm3(self.linear3(lin2)))
            sigma = morlet_wavelet(self.binorm4(self.linear4(lin2)))
        if self.fn=='mexi':
            lin1 = mexhican_hat_wavelet(self.binorm1(self.linear1(x)))
            lin2 = mexhican_hat_wavelet(self.binorm2(self.linear2(lin1)))
            Mu = mexhican_hat_wavelet(self.binorm3(self.linear3(lin2)))
            sigma = mexhican_hat_wavelet(self.binorm4(self.linear4(lin2)))
        if self.fn=='shan':
            lin1 = shannon(self.binorm1(self.linear1(x)))
            lin2 = shannon(self.binorm2(self.linear2(lin1)))
            Mu = shannon(self.binorm3(self.linear3(lin2)))
            sigma = shannon(self.binorm4(self.linear4(lin2)))
        if self.fn=='gaus':
            lin1 = gaussian_wavelet(self.binorm1(self.linear1(x)))
            lin2 = gaussian_wavelet(self.binorm2(self.linear2(lin1)))
            Mu = gaussian_wavelet(self.binorm3(self.linear3(lin2)))
            sigma = gaussian_wavelet(self.binorm4(self.linear4(lin2)))

        return Mu,sigma

class Encoder3(nn.Module):
    def __init__(self,input_no,latent_dims,fn,H2 = 10,H3=15,H4=20):
        super(Encoder3,self).__init__()
        self.linear1 = WnnLayer(input_no,H2)
        self.binorm1 = nn.BatchNorm1d(num_features = H2)
        self.linear2 = WnnLayer(H2,H3)
        self.binorm2 = nn.BatchNorm1d(num_features = H3)
        ##
        self.linear3 = WnnLayer(H3,H4)
        self.binorm3 = nn.BatchNorm1d(num_features = H4)
        self.linear4 = nn.Linear(H4,latent_dims)
        ##
        self.binorm4= nn.BatchNorm1d(num_features = latent_dims)
        self.linear5= nn.Linear(H4 , latent_dims)
        self.binorm5= nn.BatchNorm1d(num_features = latent_dims)
        self.relu =nn.ReLU()
        self.fn=fn
        
    def forward(self, x):
        if self.fn=='ggw':
            lin1 = ggw(self.binorm1(self.linear1(x)))
            lin2 = ggw(self.binorm2(self.linear2(lin1)))
            lin3 = ggw(self.binorm3(self.linear3(lin2)))
            Mu = ggw(self.binorm4(self.linear4(lin3)))
            sigma = ggw(self.binorm5(self.linear5(lin3)))
        if self.fn=='morlet':
            lin1 = morlet_wavelet(self.binorm1(self.linear1(x)))
            lin2 = morlet_wavelet(self.binorm2(self.linear2(lin1)))
            lin3 = morlet_wavelet(self.binorm3(self.linear3(lin2)))
            Mu = morlet_wavelet(self.binorm4(self.linear4(lin3)))
            sigma = morlet_wavelet(self.binorm5(self.linear5(lin3)))
        if self.fn=='mexi':
            lin1 = mexhican_hat_wavelet(self.binorm1(self.linear1(x)))
            lin2 = mexhican_hat_wavelet(self.binorm2(self.linear2(lin1)))
            lin3 = mexhican_hat_wavelet(self.binorm3(self.linear3(lin2)))
            Mu = mexhican_hat_wavelet(self.binorm4(self.linear4(lin3)))
            sigma = mexhican_hat_wavelet(self.binorm5(self.linear5(lin3)))
        if self.fn=='shan':
            lin1 = shannon(self.binorm1(self.linear1(x)))
            lin2 = shannon(self.binorm2(self.linear2(lin1)))
            lin3 = shannon(self.binorm3(self.linear3(lin2)))
            Mu = shannon(self.binorm4(self.linear4(lin3)))
            sigma = shannon(self.binorm5(self.linear5(lin3)))
        if self.fn=='gaus':
            lin1 = gaussian_wavelet(self.binorm1(self.linear1(x)))
            lin2 = gaussian_wavelet(self.binorm2(self.linear2(lin1)))
            lin3 = gaussian_wavelet(self.binorm3(self.linear3(lin2)))
            Mu = gaussian_wavelet(self.binorm4(self.linear4(lin3)))
            sigma = gaussian_wavelet(self.binorm5(self.linear5(lin3)))

        return Mu,sigma
    
class Encoder4(nn.Module):
    def __init__(self,input_no,latent_dims,fn,H2 = 10,H3=15,H4=20,H5=22):
        super(Encoder4,self).__init__()
        self.linear1 = WnnLayer(input_no,H2)
        self.binorm1 = nn.BatchNorm1d(num_features = H2)
        self.linear2 = WnnLayer(H2,H3)
        self.binorm2 = nn.BatchNorm1d(num_features = H3)
        ##
        self.linear3 = WnnLayer(H3,H4)
        self.binorm3 = nn.BatchNorm1d(num_features = H4)
        self.linear4 = nn.Linear(H4,latent_dims)
        ##
        ##
        self.linear4 = WnnLayer(H4,H5)
        self.binorm4 = nn.BatchNorm1d(num_features = H5)
        self.linear5 = nn.Linear(H5,latent_dims)
        ##
        self.binorm5= nn.BatchNorm1d(num_features = latent_dims)
        self.linear6= nn.Linear(H5 , latent_dims)
        self.binorm6= nn.BatchNorm1d(num_features = latent_dims)
        self.relu =nn.ReLU()
        self.fn=fn
        
    def forward(self, x):
        if self.fn=='ggw':
            lin1 = ggw(self.binorm1(self.linear1(x)))
            lin2 = ggw(self.binorm2(self.linear2(lin1)))
            lin3 = ggw(self.binorm3(self.linear3(lin2)))
            lin4 = ggw(self.binorm4(self.linear4(lin3)))
            Mu = ggw(self.binorm5(self.linear5(lin4)))
            sigma = ggw(self.binorm6(self.linear6(lin4)))
        if self.fn=='morlet':
            lin1 = morlet_wavelet(self.binorm1(self.linear1(x)))
            lin2 = morlet_wavelet(self.binorm2(self.linear2(lin1)))
            lin3 = morlet_wavelet(self.binorm3(self.linear3(lin2)))
            lin4 = morlet_wavelet(self.binorm4(self.linear4(lin3)))
            Mu = morlet_wavelet(self.binorm5(self.linear5(lin4)))
            sigma = morlet_wavelet(self.binorm6(self.linear6(lin4)))
        if self.fn=='mexi':
            lin1 = mexhican_hat_wavelet(self.binorm1(self.linear1(x)))
            lin2 = mexhican_hat_wavelet(self.binorm2(self.linear2(lin1)))
            lin3 = mexhican_hat_wavelet(self.binorm3(self.linear3(lin2)))
            lin4 = mexhican_hat_wavelet(self.binorm4(self.linear4(lin3)))
            Mu = mexhican_hat_wavelet(self.binorm5(self.linear5(lin4)))
            sigma = mexhican_hat_wavelet(self.binorm6(self.linear6(lin4)))
        if self.fn=='shan':
            lin1 = shannon(self.binorm1(self.linear1(x)))
            lin2 = shannon(self.binorm2(self.linear2(lin1)))
            lin3 = shannon(self.binorm3(self.linear3(lin2)))
            lin4 = shannon(self.binorm4(self.linear4(lin3)))
            Mu = shannon(self.binorm5(self.linear5(lin4)))
            sigma = shannon(self.binorm6(self.linear6(lin4)))
        if self.fn=='gaus':
            lin1 = gaussian_wavelet(self.binorm1(self.linear1(x)))
            lin2 = gaussian_wavelet(self.binorm2(self.linear2(lin1)))
            lin3 = gaussian_wavelet(self.binorm3(self.linear3(lin2)))
            lin4 = gaussian_wavelet(self.binorm4(self.linear4(lin3)))
            Mu = gaussian_wavelet(self.binorm5(self.linear5(lin4)))
            sigma = gaussian_wavelet(self.binorm6(self.linear6(lin4)))

        return Mu,sigma

In [66]:
class Decoder2(nn.Module):
    def __init__(self,latent_dims,fn,H2=15,H3=10):
        super(Decoder2, self).__init__()
#         self.linear1 = nn.Linear(latent_dims, 3400)
        self.linear1 = WnnLayer1(latent_dims,H2)
        self.binorm1 = nn.BatchNorm1d(num_features = H2)
        self.linear2 = WnnLayer1(H2,H3)
        self.binorm2 = nn.BatchNorm1d(num_features = H3)
        self.linear3 = WnnLayer1(H3,8)
        self.binorm3 = nn.BatchNorm1d(num_features = 8)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.fn=fn

    def forward(self, z):
        #print(z)
        # z = meyer_new(self.linear1(z))
        if self.fn=='ggw':
            z1 = ggw(self.binorm1(self.linear1(z)))
            linbn2 = ggw(self.binorm2(self.linear2(z1)))
            linbn3 = ggw(self.binorm3(self.linear3(linbn2)))
        if self.fn=='morlet':
            z1 = morlet_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = morlet_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = morlet_wavelet(self.binorm3(self.linear3(linbn2)))
        if self.fn=='mexi':
            z1 = mexhican_hat_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = mexhican_hat_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = mexhican_hat_wavelet(self.binorm3(self.linear3(linbn2)))
        if self.fn=='shan':
            z1 = shannon(self.binorm1(self.linear1(z)))
            linbn2 = shannon(self.binorm2(self.linear2(z1)))
            linbn3 = shannon(self.binorm3(self.linear3(linbn2)))
        if self.fn=='gaus':
            z1 = gaussian_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = gaussian_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = gaussian_wavelet(self.binorm3(self.linear3(linbn2)))

        return linbn3


class Decoder3(nn.Module):
    def __init__(self,latent_dims,fn,H2=20,H3=15,H4=10):
        super(Decoder3, self).__init__()
#         self.linear1 = nn.Linear(latent_dims, 3400)
        self.linear1 = WnnLayer1(latent_dims,H2)
        self.binorm1 = nn.BatchNorm1d(num_features = H2)
        self.linear2 = WnnLayer1(H2,H3)
        self.binorm2 = nn.BatchNorm1d(num_features = H3)
        ##
        self.linear3 = WnnLayer1(H3,H4)
        self.binorm3 = nn.BatchNorm1d(num_features = H4)
        ##
        self.linear4 = WnnLayer1(H4,8)
        self.binorm4 = nn.BatchNorm1d(num_features = 8)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.fn=fn

    def forward(self, z):
        #print(z)
        # z = meyer_new(self.linear1(z))
        if self.fn=='ggw':
            z1 = ggw(self.binorm1(self.linear1(z)))
            linbn2 = ggw(self.binorm2(self.linear2(z1)))
            linbn3 = ggw(self.binorm3(self.linear3(linbn3)))
            linbn4 = ggw(self.binorm4(self.linear4(linbn3)))
        
        if self.fn=='morlet':
            z1 = morlet_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = morlet_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = morlet_wavelet(self.binorm3(self.linear3(linbn2)))
            linbn4 = morlet_wavelet(self.binorm4(self.linear4(linbn3)))
        
        if self.fn=='mexi':
            z1 = mexhican_hat_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = mexhican_hat_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = mexhican_hat_wavelet(self.binorm3(self.linear3(linbn3)))
            linbn4 = mexhican_hat_wavelet(self.binorm4(self.linear4(linbn3)))
        
        if self.fn=='shan':
            z1 = shannon(self.binorm1(self.linear1(z)))
            linbn2 = shannon(self.binorm2(self.linear2(z1)))
            linbn3 = shannon(self.binorm3(self.linear3(linbn2)))
            linbn4 = shannon(self.binorm4(self.linear4(linbn3)))
        
        if self.fn=='gaus':
            z1 = gaussian_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = gaussian_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = gaussian_wavelet(self.binorm3(self.linear3(linbn3)))
            linbn4 = gaussian_wavelet(self.binorm4(self.linear4(linbn3)))

        return linbn4

class Decoder4(nn.Module):
    def __init__(self,latent_dims,fn,H2=22,H3=20,H4=5,H5=10):
        super(Decoder4, self).__init__()
#         self.linear1 = nn.Linear(latent_dims, 3400)
        self.linear1 = WnnLayer1(latent_dims,H2)
        self.binorm1 = nn.BatchNorm1d(num_features = H2)
        self.linear2 = WnnLayer1(H2,H3)
        self.binorm2 = nn.BatchNorm1d(num_features = H3)
        ##
        self.linear3 = WnnLayer1(H3,H4)
        self.binorm3 = nn.BatchNorm1d(num_features = H4)
        ##
        ##
        self.linear4 = WnnLayer1(H4,H5)
        self.binorm4 = nn.BatchNorm1d(num_features = H5)
        ##
        self.linear5 = WnnLayer1(H5,8)
        self.binorm5 = nn.BatchNorm1d(num_features = 8)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.fn=fn

    def forward(self, z):
        #print(z)
        # z = meyer_new(self.linear1(z))
        if self.fn=='ggw':
            z1 = ggw(self.binorm1(self.linear1(z)))
            linbn2 = ggw(self.binorm2(self.linear2(z1)))
            linbn3 = ggw(self.binorm3(self.linear3(linbn3)))
            linbn4 = ggw(self.binorm4(self.linear4(linbn3)))
            linbn5 = ggw(self.binorm5(self.linear5(linbn4)))
        
        if self.fn=='morlet':
            z1 = morlet_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = morlet_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = morlet_wavelet(self.binorm3(self.linear3(linbn2)))
            linbn4 = morlet_wavelet(self.binorm4(self.linear4(linbn3)))
            linbn5 = morlet_wavelet(self.binorm5(self.linear5(linbn4)))
        
        if self.fn=='mexi':
            z1 = mexhican_hat_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = mexhican_hat_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = mexhican_hat_wavelet(self.binorm3(self.linear3(linbn3)))
            linbn4 = mexhican_hat_wavelet(self.binorm4(self.linear4(linbn3)))
            linbn5 = mexhican_hat_wavelet(self.binorm5(self.linear5(linbn4)))
        
        if self.fn=='shan':
            z1 = shannon(self.binorm1(self.linear1(z)))
            linbn2 = shannon(self.binorm2(self.linear2(z1)))
            linbn3 = shannon(self.binorm3(self.linear3(linbn2)))
            linbn4 = shannon(self.binorm4(self.linear4(linbn3)))
            linbn5 = shannon(self.binorm5(self.linear5(linbn4)))
        
        if self.fn=='gaus':
            z1 = gaussian_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = gaussian_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = gaussian_wavelet(self.binorm3(self.linear3(linbn3)))
            linbn4 = gaussian_wavelet(self.binorm4(self.linear4(linbn3)))
            linbn5 = gaussian_wavelet(self.binorm5(self.linear5(linbn4)))
        

        return linbn5

In [67]:
class sampling_vector(nn.Module):
    def __init__(self, z,H2):
        super().__init__()
        self.z = z
        self.fc3 = nn.Linear(self.z,H2)
        self.fc_bn3 = nn.BatchNorm1d(H2)
#         self.fc4 = nn.Linear(latent_dims, H2)
#         self.fc_bn4 = nn.BatchNorm1d(H2)
        self.relu = nn.ReLU()
    def forward(self,z):
        fc3 = self.relu(self.fc_bn3(self.fc3(z)))
        return fc3

In [68]:
def get_chaotic_distribution(latent_dims,previous_seed):
  # Logistic Map
#         print("latent_dims_cha",latent_dims[1])
    noise=[]
    x0=previous_seed
#         print("prev_seed",x0)
  #print("chaotic on cards ")
  #print(latent_dims,type(latent_dims))
  #print(previous_seed)

    for i in range(latent_dims[1]):
        x1=(4.0*x0*(1-x0))
        x0=x1
#             print('x1',x1)
    #print(x0,x1)
        noise.append(x1)


    t_noise=torch.FloatTensor(noise)
#         print("t_noise",t_noise)
    return t_noise,x0

def reparameterize(mu,sigma,prev_seed):
    random_seed = 1
    torch.manual_seed(random_seed)

    std = sigma.mul(0.5).exp_()
    eps = Variable(std.data.new(std.size())).normal_()
    eps_row = eps.shape[0]
    eps_col = eps.shape[1]
#         print('eps_row',eps_row,'eps_col',eps_col)
    eps = eps.reshape(1,-1)
#         print("eps shape",eps.shape)
    chaot,prev_seed = get_chaotic_distribution(eps.shape,prev_seed)
#         print("chaot_shape",chaot.shape)
    chaot = chaot.reshape(eps_row,eps_col)
#         print("chaot_reshaped",chaot.shape)
#         print("reparameter",eps.mul(std).add_(mu))
    return chaot.mul(std).add_(mu),prev_seed

In [81]:
class VariationalAutoencoder2(nn.Module):
    def __init__(self,D_in,latent_dims,fn,H2 = 23):
        super(VariationalAutoencoder2, self).__init__()
        self.encoder = Encoder2(D_in,latent_dims,fn)
        self.decoder = Decoder2(latent_dims,fn)
        self.prev_seed =random.uniform(0,1)

    def forward(self, x):
        mu,sigma = self.encoder(x)
        prev_seed = self.prev_seed
        z,self.prev_seed = reparameterize(mu,sigma,prev_seed)
#         print("z",z.shape)
#         znew = sampling_vector(z,H)
        return self.decoder(z),mu,sigma

class VariationalAutoencoder3(nn.Module):
    def __init__(self,D_in,latent_dims,fn,H2 = 23):
        super(VariationalAutoencoder3, self).__init__()
        self.encoder = Encoder3(D_in,latent_dims,fn)
        self.decoder = Decoder3(latent_dims,fn)
        self.prev_seed =random.uniform(0,1)

    def forward(self, x):
        mu,sigma = self.encoder(x)
        prev_seed = self.prev_seed
        z,self.prev_seed = reparameterize(mu,sigma,prev_seed)
#         print("z",z.shape)
#         znew = sampling_vector(z,H)
        return self.decoder(z),mu,sigma

class VariationalAutoencoder4(nn.Module):
    def __init__(self,D_in,latent_dims,fn,H2 = 23):
        super(VariationalAutoencoder4, self).__init__()
        self.encoder = Encoder4(D_in,latent_dims,fn)
        self.decoder = Decoder4(latent_dims,fn)
        self.prev_seed =random.uniform(0,1)

    def forward(self, x):
        mu,sigma = self.encoder(x)
        prev_seed = self.prev_seed
        z,self.prev_seed = reparameterize(mu,sigma,prev_seed)
#         print("z",z.shape)
#         znew = sampling_vector(z,H)
        return self.decoder(z),mu,sigma

In [82]:
class customLoss(nn.Module):
    def __init__(self):
        super(customLoss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")

    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD

In [87]:
epochs = 1500
log_interval = 50
val_losses = []
train_losses = []
test_losses = []


def train(epoch):
    model.train()
    train_loss = 0
    
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_mse(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if epoch % 100 == 0:
            print('====> Epoch: {} Average training loss: {:.4f}'.format(
            epoch, train_loss / len(trainloader.dataset)))
        train_losses.append(train_loss / len(trainloader.dataset))
        
def test(epoch):
    with torch.no_grad():
        test_loss = 0
        for batch_idx, data in enumerate(testloader):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch = model(data)
            loss = loss_mse(recon_batch, data, mu, logvar)
            test_loss += loss.item()
            if epoch % 100 == 0:
                print('====> Epoch: {} Average test loss: {:.4f}'.format(
                    epoch, test_loss / len(testloader.dataset)))
            test_losses.append(test_loss / len(testloader.dataset))

In [88]:
loss_mse = customLoss()

In [89]:
D_in = trainloader.dataset.x.shape[1]
model = VariationalAutoencoder4(D_in,2,'shan').to(device)
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# optimizer = optim.SGD(model.parameters(), lr=0.09,momentum=0.01)
optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

In [90]:
start = time.time()
epochs=300
for epoch in range(1, epochs + 1):
    train(epoch)
#     test(epoch)
elapsed_time = time.time()-start
print("total_time taken is :",elapsed_time)
#     test(epoch)

====> Epoch: 100 Average training loss: 7.2492
====> Epoch: 200 Average training loss: 6.9918
====> Epoch: 300 Average training loss: 6.7718
total_time taken is : 21.019692420959473


In [91]:
print(train_losses[len(train_losses)-1])

6.771803571428571


## HYPER-PARAMETER TUNING

In [92]:
global model
global optimizer

model = VariationalAutoencoder4(D_in,2,'shan').to(device)
optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

In [93]:
epochs=[1800]
hl=[2,3,4]
lr=[0.0001,0.01,0.001]
mm=[0.001,0.01]
act=['morlet','ggw','mexi','shan','gaus']
opt=['adam','adagrad','sgd']
ldm=[2,3,4]

min_loss=10000
MODEL=None
BESTCOMB=None

for e in epochs:
    for h in hl:
        for l in lr:
            for m in mm:
                for a in act:
                    for o in opt:
                        for ld in ldm:
                            
                            D_in = trainloader.dataset.x.shape[1]
                            
                            if hl==2:
                                model = VariationalAutoencoder2(D_in,ld,a).to(device)
                            if hl==3:
                                model = VariationalAutoencoder3(D_in,ld,a).to(device)
                            if hl==4:
                                model = VariationalAutoencoder4(D_in,ld,a).to(device)
                            
                            if opt=='adam':
                                optimizer = optim.Adam(model.parameters(), lr=l)
                            if opt=='sgd':
                                optimizer = optim.SGD(model.parameters(), lr=l,momentum=m)
                            if opt=='adagrad':
                                optimizer = torch.optim.Adagrad(model.parameters(), lr = l)
                            
                            start = time.time()
                            for epoch in range(1, e+1):
                                train(epoch)
                            #     test(epoch)
                            elapsed_time = time.time()-start
                            #print("total_time taken is :",elapsed_time)

                            cur_loss=train_losses[len(train_losses)-1]
                            if cur_loss < min_loss:
                                min_loss=cur_loss
                                MODEL=model
                                BESTCOMB=[e,h,l,m,a,o,ld]
                            
                            
                            ## junk --ignore
                            model = VariationalAutoencoder4(D_in,2,'shan').to(device)
                            optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

====> Epoch: 100 Average training loss: 7.1512
====> Epoch: 200 Average training loss: 6.4735
====> Epoch: 300 Average training loss: 6.0957
====> Epoch: 400 Average training loss: 5.6937
====> Epoch: 500 Average training loss: 5.4797
====> Epoch: 600 Average training loss: 5.4231
====> Epoch: 700 Average training loss: 5.3323
====> Epoch: 800 Average training loss: 5.3565
====> Epoch: 900 Average training loss: 5.3477
====> Epoch: 1000 Average training loss: 5.2312
====> Epoch: 1100 Average training loss: 5.2243
====> Epoch: 1200 Average training loss: 5.2176
====> Epoch: 1300 Average training loss: 5.1454
====> Epoch: 1400 Average training loss: 5.0817
====> Epoch: 1500 Average training loss: 5.1275
====> Epoch: 1600 Average training loss: 5.0221
====> Epoch: 1700 Average training loss: 5.0038
====> Epoch: 1800 Average training loss: 4.9937
====> Epoch: 100 Average training loss: 7.2644
====> Epoch: 200 Average training loss: 6.9474
====> Epoch: 300 Average training loss: 6.7237
====

====> Epoch: 1200 Average training loss: 5.8363
====> Epoch: 1300 Average training loss: 5.7508
====> Epoch: 1400 Average training loss: 5.6639
====> Epoch: 1500 Average training loss: 5.5779
====> Epoch: 1600 Average training loss: 5.5590
====> Epoch: 1700 Average training loss: 5.5033
====> Epoch: 1800 Average training loss: 5.4564
====> Epoch: 100 Average training loss: 6.7969
====> Epoch: 200 Average training loss: 6.4447
====> Epoch: 300 Average training loss: 6.1233
====> Epoch: 400 Average training loss: 6.0588
====> Epoch: 500 Average training loss: 5.9481
====> Epoch: 600 Average training loss: 6.0075
====> Epoch: 700 Average training loss: 5.8984
====> Epoch: 800 Average training loss: 5.8720
====> Epoch: 900 Average training loss: 5.8360
====> Epoch: 1000 Average training loss: 5.7687
====> Epoch: 1100 Average training loss: 5.7790
====> Epoch: 1200 Average training loss: 5.7207
====> Epoch: 1300 Average training loss: 5.6500
====> Epoch: 1400 Average training loss: 5.5995
=

====> Epoch: 500 Average training loss: 5.8343
====> Epoch: 600 Average training loss: 5.5362
====> Epoch: 700 Average training loss: 5.3710
====> Epoch: 800 Average training loss: 5.2904
====> Epoch: 900 Average training loss: 5.2584
====> Epoch: 1000 Average training loss: 5.1675
====> Epoch: 1100 Average training loss: 5.1371
====> Epoch: 1200 Average training loss: 5.0728
====> Epoch: 1300 Average training loss: 5.0110
====> Epoch: 1400 Average training loss: 4.8963
====> Epoch: 1500 Average training loss: 4.8819
====> Epoch: 1600 Average training loss: 4.7905
====> Epoch: 1700 Average training loss: 4.8225
====> Epoch: 1800 Average training loss: 4.8120
====> Epoch: 100 Average training loss: 7.1043
====> Epoch: 200 Average training loss: 6.8402
====> Epoch: 300 Average training loss: 6.6283
====> Epoch: 400 Average training loss: 6.4847
====> Epoch: 500 Average training loss: 6.4398
====> Epoch: 600 Average training loss: 6.3771
====> Epoch: 700 Average training loss: 6.2518
====

====> Epoch: 1600 Average training loss: 5.5176
====> Epoch: 1700 Average training loss: 5.5553
====> Epoch: 1800 Average training loss: 5.5060
====> Epoch: 100 Average training loss: 7.2729
====> Epoch: 200 Average training loss: 7.0179
====> Epoch: 300 Average training loss: 6.7520
====> Epoch: 400 Average training loss: 6.4874
====> Epoch: 500 Average training loss: 6.3310
====> Epoch: 600 Average training loss: 6.1770
====> Epoch: 700 Average training loss: 6.0435
====> Epoch: 800 Average training loss: 5.9152
====> Epoch: 900 Average training loss: 5.8086
====> Epoch: 1000 Average training loss: 5.7089
====> Epoch: 1100 Average training loss: 5.6691
====> Epoch: 1200 Average training loss: 5.6323
====> Epoch: 1300 Average training loss: 5.5052
====> Epoch: 1400 Average training loss: 5.3994
====> Epoch: 1500 Average training loss: 5.3275
====> Epoch: 1600 Average training loss: 5.2690
====> Epoch: 1700 Average training loss: 5.0824
====> Epoch: 1800 Average training loss: 4.9913
=

====> Epoch: 900 Average training loss: 6.1311
====> Epoch: 1000 Average training loss: 5.9675
====> Epoch: 1100 Average training loss: 6.0048
====> Epoch: 1200 Average training loss: 5.8622
====> Epoch: 1300 Average training loss: 5.8552
====> Epoch: 1400 Average training loss: 5.8619
====> Epoch: 1500 Average training loss: 5.7905
====> Epoch: 1600 Average training loss: 5.7313
====> Epoch: 1700 Average training loss: 5.7825
====> Epoch: 1800 Average training loss: 5.6798
====> Epoch: 100 Average training loss: 7.2513
====> Epoch: 200 Average training loss: 6.5820
====> Epoch: 300 Average training loss: 6.1975
====> Epoch: 400 Average training loss: 6.0186
====> Epoch: 500 Average training loss: 5.8827
====> Epoch: 600 Average training loss: 5.7612
====> Epoch: 700 Average training loss: 5.6202
====> Epoch: 800 Average training loss: 5.5689
====> Epoch: 900 Average training loss: 5.5169
====> Epoch: 1000 Average training loss: 5.4339
====> Epoch: 1100 Average training loss: 5.3783
==

...

====> Epoch: 1700 Average training loss: 5.1374
====> Epoch: 1800 Average training loss: 5.1198
====> Epoch: 100 Average training loss: 7.2938
====> Epoch: 200 Average training loss: 6.8051
====> Epoch: 300 Average training loss: 6.5026
====> Epoch: 400 Average training loss: 6.2572
====> Epoch: 500 Average training loss: 6.1186
====> Epoch: 600 Average training loss: 5.9698
====> Epoch: 700 Average training loss: 5.8863
====> Epoch: 800 Average training loss: 5.7864
====> Epoch: 900 Average training loss: 5.7333
====> Epoch: 1000 Average training loss: 5.6409
====> Epoch: 1100 Average training loss: 5.5659
====> Epoch: 1200 Average training loss: 5.4448
====> Epoch: 1300 Average training loss: 5.4378
====> Epoch: 1400 Average training loss: 5.3940
====> Epoch: 1500 Average training loss: 5.3206
====> Epoch: 1600 Average training loss: 5.2077
====> Epoch: 1700 Average training loss: 5.1797
====> Epoch: 1800 Average training loss: 5.1181
====> Epoch: 100 Average training loss: 7.2542
==

====> Epoch: 1000 Average training loss: 5.1508
====> Epoch: 1100 Average training loss: 5.1291
====> Epoch: 1200 Average training loss: 5.1193
====> Epoch: 1300 Average training loss: 5.1481
====> Epoch: 1400 Average training loss: 5.0955
====> Epoch: 1500 Average training loss: 5.0672
====> Epoch: 1600 Average training loss: 5.0217
====> Epoch: 1700 Average training loss: 5.0483
====> Epoch: 1800 Average training loss: 4.9768
====> Epoch: 100 Average training loss: 7.1293
====> Epoch: 200 Average training loss: 6.9334
====> Epoch: 300 Average training loss: 6.6737
====> Epoch: 400 Average training loss: 6.4505
====> Epoch: 500 Average training loss: 6.3235
====> Epoch: 600 Average training loss: 6.2075
====> Epoch: 700 Average training loss: 6.1230
====> Epoch: 800 Average training loss: 6.0269
====> Epoch: 900 Average training loss: 6.0167
====> Epoch: 1000 Average training loss: 5.8901
====> Epoch: 1100 Average training loss: 5.8026
====> Epoch: 1200 Average training loss: 5.6618
=

====> Epoch: 300 Average training loss: 6.5586
====> Epoch: 400 Average training loss: 6.4426
====> Epoch: 500 Average training loss: 6.2941
====> Epoch: 600 Average training loss: 6.1259
====> Epoch: 700 Average training loss: 6.0872
====> Epoch: 800 Average training loss: 5.9727
====> Epoch: 900 Average training loss: 5.8963
====> Epoch: 1000 Average training loss: 5.9264
====> Epoch: 1100 Average training loss: 5.8520
====> Epoch: 1200 Average training loss: 5.7812
====> Epoch: 1300 Average training loss: 5.6916
====> Epoch: 1400 Average training loss: 5.6265
====> Epoch: 1500 Average training loss: 5.6057
====> Epoch: 1600 Average training loss: 5.5403
====> Epoch: 1700 Average training loss: 5.5459
====> Epoch: 1800 Average training loss: 5.5024
====> Epoch: 100 Average training loss: 7.0875
====> Epoch: 200 Average training loss: 6.7481
====> Epoch: 300 Average training loss: 6.4658
====> Epoch: 400 Average training loss: 6.3911
====> Epoch: 500 Average training loss: 6.3090
====

====> Epoch: 1400 Average training loss: 5.5865
====> Epoch: 1500 Average training loss: 5.5085
====> Epoch: 1600 Average training loss: 5.4021
====> Epoch: 1700 Average training loss: 5.4402
====> Epoch: 1800 Average training loss: 5.3155
====> Epoch: 100 Average training loss: 7.2053
====> Epoch: 200 Average training loss: 6.5922
====> Epoch: 300 Average training loss: 6.3541
====> Epoch: 400 Average training loss: 6.0177
====> Epoch: 500 Average training loss: 5.9269
====> Epoch: 600 Average training loss: 5.6380
====> Epoch: 700 Average training loss: 5.6083
====> Epoch: 800 Average training loss: 5.4580
====> Epoch: 900 Average training loss: 5.3493
====> Epoch: 1000 Average training loss: 5.3257
====> Epoch: 1100 Average training loss: 5.2641
====> Epoch: 1200 Average training loss: 5.2734
====> Epoch: 1300 Average training loss: 5.2144
====> Epoch: 1400 Average training loss: 5.1709
====> Epoch: 1500 Average training loss: 5.2045
====> Epoch: 1600 Average training loss: 5.2101
=

====> Epoch: 700 Average training loss: 5.7988
====> Epoch: 800 Average training loss: 5.7290
====> Epoch: 900 Average training loss: 5.6351
====> Epoch: 1000 Average training loss: 5.6006
====> Epoch: 1100 Average training loss: 5.5660
====> Epoch: 1200 Average training loss: 5.5404
====> Epoch: 1300 Average training loss: 5.5281
====> Epoch: 1400 Average training loss: 5.4758
====> Epoch: 1500 Average training loss: 5.4993
====> Epoch: 1600 Average training loss: 5.3850
====> Epoch: 1700 Average training loss: 5.4381
====> Epoch: 1800 Average training loss: 5.4048
====> Epoch: 100 Average training loss: 7.2212
====> Epoch: 200 Average training loss: 7.0154
====> Epoch: 300 Average training loss: 6.6700
====> Epoch: 400 Average training loss: 6.2253
====> Epoch: 500 Average training loss: 5.9735
====> Epoch: 600 Average training loss: 5.8473
====> Epoch: 700 Average training loss: 5.7031
====> Epoch: 800 Average training loss: 5.6414
====> Epoch: 900 Average training loss: 5.5990
====

In [94]:
BESTCOMB

[1800, 4, 0.01, 0.001, 'mexi', 'adam', 3]

In [95]:
MODEL

VariationalAutoencoder4(
  (encoder): Encoder4(
    (linear1): WnnLayer()
    (binorm1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear2): WnnLayer()
    (binorm2): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear3): WnnLayer()
    (binorm3): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear4): WnnLayer()
    (binorm4): BatchNorm1d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear5): Linear(in_features=22, out_features=2, bias=True)
    (binorm5): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear6): Linear(in_features=22, out_features=2, bias=True)
    (binorm6): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (decoder): Decoder4(
    (linear1): WnnLayer1()
    (binorm1): BatchNorm1d(22, eps=1e-05, momentum=0.1, affine=True, track_ru

In [96]:
x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)


In [97]:
x_hat

[tensor([[-0.3118,  0.1687,  0.2261,  ...,  1.3255,  0.8761,  0.9793],
         [ 0.4720,  0.0446,  0.9238,  ...,  1.0182,  0.7046,  0.6949],
         [ 0.4692,  1.0398,  0.6773,  ...,  0.7950,  0.7351,  0.7032],
         ...,
         [ 0.9159,  0.5382, -0.8531,  ...,  0.9325,  0.4943,  0.4635],
         [ 0.3632, -1.0234,  0.7830,  ..., -0.2928,  0.8445,  0.8321],
         [ 0.3698,  0.3809, -0.4323,  ...,  0.9730,  0.4016,  0.4400]])]

In [98]:
d1 = pd.DataFrame(np.array(x_hat[0]))

In [99]:
d1

,0,1,2,3,4,5,6,7
0,-0.311789,0.168675,0.226055,-0.032891,0.051957,1.325491,0.876147,0.979317
1,0.472000,0.044634,0.923827,0.220941,-0.088096,1.018224,0.704592,0.694888
2,0.469209,1.039768,0.677300,0.219050,0.333319,0.794968,0.735060,0.703188
3,0.714625,1.005984,-0.329259,0.366561,-1.024209,1.063908,0.257516,0.174657
4,-1.015051,0.140409,1.064925,-1.013332,0.002985,0.591028,-0.054921,-0.060371
...,...,...,...,...,...,...,...,...
6995,0.813924,0.183120,0.388053,0.441437,1.058147,0.661996,0.699850,0.669020
6996,0.883374,0.516208,0.776105,0.505846,-0.232706,0.869419,0.412233,0.383127
6997,0.915903,0.538235,-0.853066,0.517998,-0.090296,0.932478,0.494276,0.463474
6998,0.363226,-1.023427,0.782989,0.160454,0.376672,-0.292759,0.844490,0.832078


In [100]:
# d2

In [101]:
org =pd.DataFrame(np.array(trainloader.dataset.x))

In [102]:
# pd.DataFrame(np.array(testloader.dataset.x)).mean()


In [103]:
org.std()

0    0.192962
1    0.141110
2    0.287587
3    0.248447
4    0.192987
5    0.456147
6    0.499946
7    0.286641
dtype: float32

In [104]:
syn_test = pd.concat([d1],ignore_index = True)

In [105]:
syn_test.shape

(7000, 8)

In [106]:
syn_test.std()

0    0.511541
1    0.462386
2    0.641109
3    0.457863
4    0.601014
5    0.534221
6    0.558559
7    0.556318
dtype: float32

In [107]:
# d1.to_csv('vae-wnn-yes.csv')

## Evasion Attack

In [110]:
x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(testloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)

d1 = pd.DataFrame(np.array(x_hat[0]))
eva_test = pd.concat([d1],ignore_index = True)

In [111]:
DATA_PATH = "../../../datasets/Churn_Modelling.csv"

## Dataset Overview

df_base = pd.read_csv(DATA_PATH, sep=',')
df_base.head()

cols = df_base.columns

df = pd.read_csv(DATA_PATH, sep=',')
# replace nan with -99
y = df['Exited']
df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
   'Gender','Exited'],axis=1,inplace=True)
#df = df.fillna(-99)
#df = df.fillna(-99)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data=scaler.fit_transform(df)
df1=pd.DataFrame(data)
# randomly split
X_train, X_test,Y_train,Y_test = train_test_split(df1, y,test_size=0.3, stratify=y, random_state=4)


Y_train.reset_index(drop = True);Y_test.reset_index(drop = True)

## Models

## Logistic Regression

print("LOGISTIC REGRESSION")

np.random.seed(42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train,Y_train)

preds = lr.predict(eva_test)

#roc_auc_score(Y_test,ypred_lr)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)

print("Decision Tree")

np.random.seed(42)

from sklearn.tree import  DecisionTreeClassifier

logi = DecisionTreeClassifier()
logi.fit(X_train,Y_train)

preds =logi.predict(eva_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)


print("LGBM Classifier")

np.random.seed(42)

from lightgbm import LGBMClassifier

# parameters = {'n_estimators': [25,50,100,150,175,200]}
# parameters = {'criterion' : ["gini", "entropy"],
#               'n_estimators': [25,50,100,150,175,200]}
# best_model=None
# best_score=0.0

# for ne in parameters['n_estimators']:
#     gb = LGBMClassifier(n_estimators=ne)
#     gb.fit(X_train,Y_train)
#     y_predg=gb.predict(X_test)

#     accuracy=accuracy_score(Y_test,y_predg)
#     auc=roc_auc_score(Y_test,y_predg)
#     # print('Combination',c,ne)
#     #print('AUC',auc)

#     if auc > best_score:
#         best_model=gb
#         best_score=auc
        
gb = LGBMClassifier(n_estimators=150)
gb.fit(X_train,Y_train)
preds=gb.predict(eva_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



print("XGB Classifier")

from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train,Y_train)
preds = xgb_model.predict(eva_test)


from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



LOGISTIC REGRESSION
2389 0 611 0
0.0
1.0
0.5
Decision Tree
1845 544 478 133
0.2176759410801964
0.7722896609460025
0.49498280101309944
LGBM Classifier
2333 56 601 10
0.016366612111292964
0.9765592298032649
0.49646292095727895
XGB Classifier
2321 68 597 14
0.022913256955810146
0.9715362076182503
0.49722473228703024


## Poison Attack

In [112]:
x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)

d1 = pd.DataFrame(np.array(x_hat[0]))
eva_train = pd.concat([d1],ignore_index = True)

In [113]:
DATA_PATH = "../../../datasets/Churn_Modelling.csv"

## Dataset Overview

df_base = pd.read_csv(DATA_PATH, sep=',')
df_base.head()

cols = df_base.columns

df = pd.read_csv(DATA_PATH, sep=',')
# replace nan with -99
y = df['Exited']
df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
   'Gender','Exited'],axis=1,inplace=True)
#df = df.fillna(-99)
#df = df.fillna(-99)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data=scaler.fit_transform(df)
df1=pd.DataFrame(data)
# randomly split
X_train, X_test,Y_train,Y_test = train_test_split(df1, y,test_size=0.3, random_state=4)


Y_train.reset_index(drop = True);Y_test.reset_index(drop = True)

## Models

## Logistic Regression

print("LOGISTIC REGRESSION")

np.random.seed(42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(eva_train,Y_train)

preds = lr.predict(X_test)

#roc_auc_score(Y_test,ypred_lr)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)

print("Decision Tree")

np.random.seed(42)

from sklearn.tree import  DecisionTreeClassifier

logi = DecisionTreeClassifier()
logi.fit(eva_train,Y_train)

preds =logi.predict(X_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)


print("LGBM Classifier")

np.random.seed(42)

from lightgbm import LGBMClassifier

# parameters = {'n_estimators': [25,50,100,150,175,200]}
# parameters = {'criterion' : ["gini", "entropy"],
#               'n_estimators': [25,50,100,150,175,200]}
# best_model=None
# best_score=0.0

# for ne in parameters['n_estimators']:
#     gb = LGBMClassifier(n_estimators=ne)
#     gb.fit(X_train,Y_train)
#     y_predg=gb.predict(X_test)

#     accuracy=accuracy_score(Y_test,y_predg)
#     auc=roc_auc_score(Y_test,y_predg)
#     # print('Combination',c,ne)
#     #print('AUC',auc)

#     if auc > best_score:
#         best_model=gb
#         best_score=auc
        
gb = LGBMClassifier(n_estimators=150)
gb.fit(eva_train,Y_train)
preds=gb.predict(X_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



print("XGB Classifier")

from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb_model = xgb.fit(eva_train,Y_train)
preds = xgb_model.predict(X_test)


from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



LOGISTIC REGRESSION
2416 0 584 0
0.0
1.0
0.5
Decision Tree
1474 942 327 257
0.4400684931506849
0.6100993377483444
0.5250839154495146
LGBM Classifier
2377 39 577 7
0.011986301369863013
0.9838576158940397
0.4979219586319514
XGB Classifier
1954 462 508 76
0.13013698630136986
0.8087748344370861
0.46945591036922796
